### 大模型学习笔记




In [ ]:
#pytorch降级

!pip uninstall torch torchvision torchaudio -y

!pip install torch torchvision torchaudio \
            -i https://mirrors.aliyun.com/pypi/simple/ \
            --extra-index-url https://download.pytorch.org/whl/cu12x


In [ ]:
# 环境安装
!pip install . --no-build-isolation --no-deps  

In [ ]:
# 监控安装
! pip install nvitop && pip install nvidia-ml-py -U


In [16]:
# 环境信息
!pwd
!llamafactory-cli version
!nvidia-smi

/mnt/workspace/LLaMA-Factory
/usr/local/lib/python3.11/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
----------------------------------------------------------
| Welcome to LLaMA Factory, version 0.9.4.dev0           |
|                                                        |
| Project page: https://github.com/hiyouga/LLaMA-Factory |
----------------------------------------------------------
Tue Dec  2 22:17:34 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id 

# 监控显卡
`nvitop -m auto`

`nvitop -m compact`

`nvitop -m full`

# 微调--〉节约成本

train_loss: 越小越好

In [ ]:
# 训练
!export USE_MODELSCOPE_HUB=1 && llamafactory-cli train examples/train_lora/llama3_lora_sft.yaml

In [58]:
# 训练+评估
!export USE_MODELSCOPE_HUB=1 && llamafactory-cli eval examples/train_lora/llama3_lora_eval.yaml

/usr/local/lib/python3.11/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
Traceback (most recent call last):
  File "/usr/local/bin/llamafactory-cli", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/mnt/workspace/LLaMA-Factory/src/llamafactory/cli.py", line 24, in main
    launcher.launch()
  File "/mnt/workspace/LLaMA-Factory/src/llamafactory/launcher.py", line 147, in launch
    raise NotImplementedError("Evaluation will be deprecated in the future.")
NotImplementedError: Evaluation will be deprecated in the future.


In [ ]:
# chat（原始推理）
!cd LLaMA-Factory && export USE_MODELSCOPE_HUB=1 && llamafactory-cli chat examples/inference/llama3_lora_sft.yaml

In [ ]:
# 合并导出
!export USE_MODELSCOPE_HUB=1 && llamafactory-cli export examples/merge_lora/llama3_lora_sft.yaml


### 推理
#### 合并后的推理
```
cd LLaMA-Factory && export USE_MODELSCOPE_HUB=1 && llamafactory-cli chat examples/inference/llama3_lora_sft-merge.yaml
```

#### 开端口推理
```

cd LLaMA-Factory

export USE_MODELSCOPE_HUB=1 

# readme上的显存不够
#API_PORT=8000 llamafactory-cli api examples/inference/llama3_lora_sft-merge.yaml infer_backend=vllm vllm_enforce_eager=true

API_PORT=8000 CUDA_VISIBLE_DEVICES=0 llamafactory-cli api examples/inference/llama3_lora_sft-merge.yaml

```


In [ ]:
# python 批量调端口推理

from openai import OpenAI
client = OpenAI(api_key="0", base_url="http://0.0.0.0:8000/v1")
messages = [{"role": "user", "content": "你是谁?"}]
# LLM-Research/Meta-Llama-3-8B-Instruct
# LLM-Research/llama3_lora_sft
result = client.chat.completions.create(messages=messages, model="llama3_lora_sft")
print(result)


messages = [{"role": "user", "content": "解释人工智能中的迁移学习"}]
# LLM-Research/Meta-Llama-3-8B-Instruct
# LLM-Research/llama3_lora_sft
result = client.chat.completions.create(messages=messages, model="llama3_lora_sft")
print(result)



In [ ]:
#通过vllm进行推理

python scripts/vllm_infer.py --model_name_or_path  /mnt/workspace/LLaMA-Factory/output/llama3_lora_sft
 --dataset alpaca_zh_demo

# 模型量化导出

### 压缩最终微调后的模型大小，精度丢失换取gpu耗小

### 量化分类




In [ ]:
!pip install 

In [ ]:
# 量化安装gpt库

!pip install https://github.com/ModelCloud/GPTQModel/releases/download/v4.2.5/gptqmodel-4.2.5+cu12.torch2.8-cp311-cp311-linux_x86_64.whl
!pip install -r gptmodel_requires.txt

In [ ]:
# 量化

! export USE_MODELSCOPE_HUB=1 && llamafactory-cli export examples/merge_lora/llama3_gptq.yaml

### 上一步量化出错

from transformers import AutoConfig
config = AutoConfig.from_pretrained("/mnt/workspace/LLaMA-Factory/output/llama3_lora_sft")
print("hidden_size:", config.hidden_size)
print("num_attention_heads:", config.num_attention_heads)
print("num_key_value_heads:", config.num_key_value_heads)
print("head_dim:", config.hidden_size // config.num_attention_heads)
print("rope_dim:", getattr(config, "rope_scaling", None))


# DDP并行训练

- 单机多卡
- 多机多卡


In [18]:
#ddp 训练
! export USE_MODELSCOPE_HUB=1 && FORCE_TORCHRUN=1 DISABLE_VERSION_CHECK=1 llamafactory-cli train examples/train_lora/llama4_lora_sft_ds3.yaml


/usr/local/lib/python3.11/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
[INFO|2025-12-02 22:18:24] llamafactory.launcher:143 >> Initializing 1 distributed tasks at: 127.0.0.1:56721
/usr/local/lib/python3.11/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
/usr/local/lib/python3.11/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that instal